<center><img src = "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.WmN3Js3CIU7Yy072-JKSEQHaDH%26pid%3DApi&f=1" width = 400></center>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

<center>Toronto is a beautiful city with many diverse neighborhoods. I will explore analyze them by scraping postal codes from Wikipedia and utilizing Foursquare's powerful API to derive additional insights.</center>

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

!pip install geocoder
import geocoder

!pip install bs4
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


# 1. Scraping Postal Codes from Wikipedia

In [2]:
# Scraping text from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
soup = BeautifulSoup(requests.get(url).text, 'html5lib')
tables = soup.find_all('table')
len(tables)  # Checking how many tables there are

3

In [3]:
# Getting details about the tables to identify the correct one
for i, table in enumerate(tables):
    print('Table ', i)
    print(table.attrs)
    print('# rows: ', len(table.find_all('tr')))
    print('# columns: ', len(table.tr.find_all('td')), '\n')

Table  0
{'style': 'width:100%; border-collapse:collapse; border:1px solid #ccc;', 'cellpadding': '2', 'cellspacing': '0', 'rules': 'all'}
# rows:  20
# columns:  9 

Table  1
{'class': ['navbox']}
# rows:  4
# columns:  2 

Table  2
{'cellspacing': '0', 'style': 'background-color: #F8F8F8;', 'width': '100%'}
# rows:  2
# columns:  12 



In [4]:
# Getting postal codes from the first table
pcTable = []  # Empty list to store table content

for cell in tables[0].find_all('td'):
    pcRow = {}
    if cell.span.text == 'Not assigned':  # Removing unnassigned codes
        pass
    else:
        pcRow['PostalCode'] = cell.p.text[:3]
        pcRow['Borough'] = (cell.span.text).split('(')[0]
        pcRow['Neighborhood'] = (((((cell.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        pcTable.append(pcRow)

df = pd.DataFrame(pcTable)  # Creating dataframe
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade': 'Downtown Toronto Stn A',  # Cleaning text
                                       'East TorontoBusiness reply mail Processing Centre969 Eastern': 'East Toronto Business',
                                       'EtobicokeNorthwest': 'Etobicoke Northwest','East YorkEast Toronto': 'East York/East Toronto',
                                       'MississaugaCanada Post Gateway Processing Centre': 'Mississauga'})

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
print('DataFrame size: ', df.shape)

DataFrame size:  (103, 3)


# 2. Collecting Latitude and Longitude for Toronto Neighborhoods

Could never get geocoder to return a response, so I had to rely on the CSV provided in the hints.

In [8]:
gs_data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
gs_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# Joining the data from the csv with the original dataframe on the Postal Code column
df = df.set_index('PostalCode').join(gs_data.set_index('Postal Code'))
df.reset_index(inplace =True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# 3. Extracting Toronto Data From Foursquare API


# 4. Neighborhood Analysis - Segmenting and Clustering